In [94]:
from collections import defaultdict
from datetime import datetime
import json
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))

from config import (
    FILTERED_DATA_PATH,
    TRAIN_DATA_PATH,
    VAL_DATA_PATH,
    TEST_DATA_PATH,
    METADATA_PATH,
    EMBEDDINGS_TITLE_PATH,
    EMBEDDINGS_DESCR_PATH,
    EMBEDDINGS_COMBI_PATH,
    RANDOM_STATE,
    N_RECOMMENDATIONS,
    N_EPOCHS,
    RECOMMENDATIONS_PATH,
    TRANSFORMED_DATA_PATH,
    SPLIT_DATE_TRAIN_VAL,
    SPLIT_DATE_VAL_TEST,
)
import utils.utils as utils

In [95]:
# importing training data
train_df = pd.read_parquet("../" + TRAIN_DATA_PATH)

# importing validation data
val_df = pd.read_parquet("../" + VAL_DATA_PATH)

# importing test data
test_df = pd.read_parquet("../" + TEST_DATA_PATH)

# importing metadata
meta_df = pd.read_parquet("../" + METADATA_PATH)

# importing the transformed data
transformed_df = pd.read_parquet("../" + TRANSFORMED_DATA_PATH)

# importing the filtered data
filtered_df = pd.read_parquet("../" + FILTERED_DATA_PATH)

# importing title embeddings
title_embeddings = pd.read_parquet("../" + EMBEDDINGS_TITLE_PATH)

# importing description embeddings
descr_embeddings = pd.read_parquet("../" + EMBEDDINGS_DESCR_PATH)

In [64]:
train_w_meta = train_df.merge(meta_df, how="left", on="prd_number")

In [92]:
title_embeddings[title_embeddings["episode"] == "11031452096"].iloc[:, 1:].values.flatten()

array([ 6.41332484e-02,  7.79344138e-02, -2.64630157e-02,  5.23740529e-03,
        1.38073446e-01, -4.25269213e-02,  8.05566792e-02, -6.78167210e-02,
       -5.34091274e-02,  3.88904377e-02,  1.18827171e-01, -1.08937493e-02,
        3.64053496e-02, -2.37905213e-03, -1.07288580e-02,  1.47286632e-02,
       -3.11938640e-02, -1.83801236e-02,  1.01044264e-02,  3.26254898e-02,
       -3.15338124e-02, -1.12534519e-02, -2.07916544e-02,  4.66829197e-03,
       -1.29991516e-01, -2.77478147e-02, -9.61893538e-02, -5.78676419e-02,
       -1.45617323e-02, -4.02035027e-02,  6.33498978e-02, -1.19316817e-02,
       -3.72808327e-02,  5.99347151e-02, -2.62064691e-02,  3.82109392e-02,
        1.30096786e-02, -5.52613904e-02,  6.60928456e-02,  3.89208883e-02,
       -3.48054611e-02,  2.56093373e-02,  6.53618744e-02,  1.66150560e-02,
       -5.93057742e-02,  3.34925341e-02, -4.09815259e-02, -2.51467783e-02,
       -1.30924101e-02, -4.76182806e-02, -9.87955974e-03, -3.30856197e-02,
       -4.92432016e-02, -

In [86]:
import pandas as pd

# Load your data
df = pd.read_csv("../results/cf_experiments.csv", header=0)  # or manually create the DataFrame
print(df)
# Select only the independent variables
df.columns = df.columns.str.strip()

X = df[['n_components', 'damping', 'reg']]

# Compute pairwise Pearson correlations
correlation_matrix = X.corr()

print(correlation_matrix)


     n_components   damping    reg   ndcg@10
0              50         1  0.050  0.010545
1              50         1  0.100  0.002846
2              50         1  0.150  0.000797
3              50         5  0.050  0.017417
4              50         5  0.100  0.006394
..            ...       ...    ...       ...
103           800       100  0.003  0.068805
104           800       100  0.004  0.069434
105           800       120  0.002  0.067800
106           800       120  0.003  0.069293
107           800       120  0.004  0.069966

[108 rows x 4 columns]
              n_components   damping       reg
n_components      1.000000  0.930548 -0.614693
damping           0.930548  1.000000 -0.620835
reg              -0.614693 -0.620835  1.000000
